# 🚀 Homework 9: Text Generation & HuggingFace Spaces
**MIS 769 - Big Data Analytics for Business | Spring 2026**

**Points:** 20 | **Due:** Sunday, April 12, 2026 @ 11pm Pacific

**Author:** Richard Young, Ph.D. | UNLV Lee Business School

**Compute:** GPU recommended — Spaces deployment is free

---

## What You'll Learn

1. Use HuggingFace Transformers for text generation
2. Understand generation parameters (temperature, top-k, top-p)
3. Build an interactive demo with Gradio
4. Deploy your application to HuggingFace Spaces

---

## How GPT-2 Works: The Attention Mechanism

GPT-2 is built on the **Transformer architecture**, which uses **self-attention** to process text. Instead of reading words sequentially like older models (RNNs), attention allows the model to look at all words simultaneously and decide which ones are most relevant for predicting the next word.

attention_mechanism.svg

**Key concepts in the diagram above:**
- **Query (Q)**: What the model is currently trying to predict
- **Key (K)**: What information each token provides for matching
- **Value (V)**: The actual content to retrieve when there's a match
- **Multi-Head Attention**: Running multiple attention computations in parallel to capture different relationships

---

## Part 1: Text Generation (4 points)

In [ ]:
!pip install transformers torch gradio -q

from transformers import pipeline

# Load GPT-2 (small, works on CPU)
generator = pipeline("text-generation", model="gpt2")

print("✅ GPT-2 model loaded")

In [ ]:
# Basic text generation
prompt = "The future of artificial intelligence"

output = generator(
    prompt,
    max_new_tokens=50,
    do_sample=True,
    pad_token_id=50256  # GPT-2 fix
)

print("📝 TEXT GENERATION")
print("=" * 60)
print(f"Prompt: \"{prompt}\"")
print(f"\nGenerated:")
print(output[0]['generated_text'])

## Part 2: Parameter Exploration (4 points)

In [ ]:
# Temperature comparison
print("📊 TEMPERATURE COMPARISON")
print("=" * 60)

prompt = "Once upon a time"
temperatures = [0.3, 0.7, 1.2]

for temp in temperatures:
    output = generator(
        prompt,
        max_new_tokens=30,
        temperature=temp,
        do_sample=True,
        pad_token_id=50256
    )
    generated = output[0]['generated_text'][len(prompt):]
    print(f"\nTemperature {temp}:")
    print(f"  {generated[:80]}...")

In [ ]:
# Top-k and Top-p comparison
print("📊 TOP-K AND TOP-P COMPARISON")
print("=" * 60)

configs = [
    {"top_k": 10, "top_p": 1.0, "name": "top_k=10"},
    {"top_k": 50, "top_p": 1.0, "name": "top_k=50"},
    {"top_k": 0, "top_p": 0.9, "name": "top_p=0.9"},
    {"top_k": 0, "top_p": 0.5, "name": "top_p=0.5"},
]

for cfg in configs:
    output = generator(
        prompt,
        max_new_tokens=30,
        top_k=cfg["top_k"] if cfg["top_k"] > 0 else None,
        top_p=cfg["top_p"],
        do_sample=True,
        pad_token_id=50256
    )
    generated = output[0]['generated_text'][len(prompt):]
    print(f"\n{cfg['name']}:")
    print(f"  {generated[:80]}...")

In [ ]:
# Create a summary table
print("\n📊 PARAMETER SUMMARY")
print("=" * 60)
print("""
| Parameter   | Effect                          | Typical Values |
|-------------|--------------------------------|----------------|
| temperature | Higher = more random           | 0.3-1.0        |
| top_k       | Limits token choices           | 20-100         |
| top_p       | Nucleus sampling threshold     | 0.8-0.95       |
| max_tokens  | Maximum output length          | 50-500         |
""")

## Part 3: Gradio Interface (5 points)

In [ ]:
import gradio as gr

def generate_text(prompt, temperature, max_length):
    """Generate text with specified parameters."""
    if not prompt:
        return "Please enter a prompt."
    
    result = generator(
        prompt,
        max_new_tokens=int(max_length),
        temperature=temperature,
        do_sample=True,
        pad_token_id=50256
    )
    return result[0]['generated_text']

# Create Gradio interface
demo = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(label="Prompt", placeholder="Enter your prompt here...", lines=2),
        gr.Slider(0.1, 1.5, value=0.7, step=0.1, label="Temperature"),
        gr.Slider(20, 200, value=50, step=10, label="Max New Tokens")
    ],
    outputs=gr.Textbox(label="Generated Text", lines=5),
    title="🚀 Text Generator",
    description="Generate creative text using GPT-2. Adjust temperature for creativity.",
    examples=[
        ["The future of technology is", 0.7, 50],
        ["Once upon a time in a magical forest", 0.9, 100],
        ["Dear customer, thank you for", 0.5, 50]
    ]
)

print("🎨 GRADIO INTERFACE CREATED")
print("=" * 60)
print("Run demo.launch() to start the interface")

In [ ]:
# Launch the demo (in Colab, this creates a public URL)
demo.launch(share=True)

## Part 4: HuggingFace Spaces Deployment (5 points)

In [ ]:
# Generate the app.py file for Spaces
app_code = '''
import gradio as gr
from transformers import pipeline

# Load model
generator = pipeline("text-generation", model="gpt2")

def generate_text(prompt, temperature, max_length):
    if not prompt:
        return "Please enter a prompt."
    
    result = generator(
        prompt,
        max_new_tokens=int(max_length),
        temperature=temperature,
        do_sample=True,
        pad_token_id=50256
    )
    return result[0]['generated_text']

demo = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(label="Prompt", placeholder="Enter your prompt..."),
        gr.Slider(0.1, 1.5, value=0.7, label="Temperature"),
        gr.Slider(20, 200, value=50, label="Max Tokens")
    ],
    outputs=gr.Textbox(label="Generated Text"),
    title="Text Generator",
    description="Generate text with GPT-2"
)

demo.launch()
'''

print("📄 APP.PY FOR HUGGINGFACE SPACES")
print("=" * 60)
print(app_code)

In [ ]:
# Generate requirements.txt
requirements = """transformers
gradio
torch
"""

print("📄 REQUIREMENTS.TXT")
print("=" * 60)
print(requirements)

### Deployment Steps

1. Go to [huggingface.co/spaces](https://huggingface.co/spaces)
2. Click "Create new Space"
3. Choose a name (e.g., `text-generator`)
4. Select **Gradio** as the SDK
5. Upload `app.py` and `requirements.txt`
6. Wait for the build to complete
7. **Copy your Space URL and paste it below!**

In [ ]:
# Record your Space URL
space_url = "https://huggingface.co/spaces/YOUR_USERNAME/YOUR_SPACE_NAME"

print("🚀 YOUR HUGGINGFACE SPACE")
print("=" * 60)
print(f"URL: {space_url}")
print("\n⚠️ Replace with your actual Space URL before submitting!")

## Part 5: Documentation (2 points)

In [ ]:
# Test various generation scenarios
test_prompts = [
    "Write a professional email starting with: Dear hiring manager,",
    "Create a product description for: A smart water bottle that",
    "Generate a tweet about: The benefits of remote work"
]

print("📊 GENERATION SHOWCASE")
print("=" * 60)

for prompt in test_prompts:
    output = generator(
        prompt,
        max_new_tokens=50,
        temperature=0.7,
        do_sample=True,
        pad_token_id=50256
    )
    print(f"\nPrompt: {prompt[:50]}...")
    print(f"Output: {output[0]['generated_text'][len(prompt):][:100]}...")

---

## Questions to Answer

**Q1:** How do temperature and top-p affect output creativity?

*Your answer:*

**Q2:** What trade-offs did you make for Spaces deployment?

*Your answer:*

**Q3:** What would you add to make this production-ready?

*Your answer:*

**Q4:** Your HuggingFace Spaces URL:

*Your URL:*

---

## Submission Checklist

| Item | Points | Done? |
|------|--------|-------|
| Part 1: Text generation | 4 | ☐ |
| Part 2: Parameter exploration | 4 | ☐ |
| Part 3: Gradio interface | 5 | ☐ |
| Part 4: Spaces deployment | 5 | ☐ |
| Part 5: Documentation | 2 | ☐ |
| **Total** | **20** | |

**Don't forget to include your HuggingFace Spaces URL!**